# MCP Tutorial - Jupyter Notebook Demo

This notebook demonstrates how to use the MCP servers with **direct function calls** in Jupyter/Colab.

⚠️ **Note:** The MCP orchestrator with subprocesses does NOT work in Jupyter/Colab. For OpenAI gpt-5-nano integration, use `interactive_client.py` in regular Python.

## Setup

First, make sure you've installed the requirements:

In [ ]:
# Install requirements (only need to run once)
!pip install -q -r requirements.txt

## Direct Function Calls

All server functions can be imported and called directly as regular Python functions.
This is the recommended way to use MCP servers in Jupyter/Colab notebooks.

In this section, we'll explore how each server's functions work, including:
- **Successful queries** - When data exists in the knowledge base
- **Error responses** - What hints the LLM receives when data is not found

These error responses contain valuable guidance that helps gpt-5-nano recover from failures and choose alternative approaches.

### Ticket Server Functions

The ticket server manages support tickets with search, detail lookup, metrics, and similarity functions.
Let's see both successful queries and error handling.

In [ ]:
# Import ticket server functions
# For Google Colab: files are uploaded to root directory (no 'servers' folder)
from ticket_server import search_tickets, get_ticket_details, get_ticket_metrics, find_similar_tickets_to, TICKETS

# Example 1: Search for critical priority tickets (SUCCESS)
print("=" * 60)
print("Example 1: Searching for critical tickets")
print("=" * 60)
critical_tickets = search_tickets(priority="critical")
print(f"Found {critical_tickets['total_count']} critical tickets:\n")
for ticket in critical_tickets['tickets']:
    print(f"  {ticket['ticket_id']}: {ticket['subject']}")

In [ ]:
# Example 2: Get details of an existing ticket (SUCCESS)
print("\n" + "=" * 60)
print("Example 2: Getting details for existing ticket TKT-1001")
print("=" * 60)
ticket = get_ticket_details("TKT-1001")
print(f"Ticket: {ticket['ticket_id']}")
print(f"Subject: {ticket['subject']}")
print(f"Status: {ticket['status']}")
print(f"Priority: {ticket['priority']}")
print(f"Description: {ticket['description'][:100]}...")

# Example 3: Try to get a non-existent ticket (ERROR WITH HINTS)
print("\n" + "=" * 60)
print("Example 3: Attempting to get non-existent ticket TKT-9999")
print("=" * 60)
error_response = get_ticket_details("TKT-9999")
print("This ticket doesn't exist. Here's the error response with LLM hints:\n")
import json
print(json.dumps(error_response, indent=2))
print("\n💡 Notice the 'suggested_actions' and 'follow_up_tools' that guide the LLM!")

In [ ]:
# Example 4: Get ticket metrics (SUCCESS)
print("\n" + "=" * 60)
print("Example 4: Getting ticket metrics for last 7 days")
print("=" * 60)
metrics = get_ticket_metrics("last_7_days")
print(f"Ticket Metrics (Last 7 Days):")
print(f"  Total: {metrics['total_tickets']}")
print(f"  Open: {metrics['open_tickets']}")
print(f"  In Progress: {metrics['in_progress_tickets']}")
print(f"  Resolved: {metrics['resolved_tickets']}")
print(f"  Avg Resolution Time: {metrics['avg_resolution_time_hours']} hours")

### Customer Server Functions

The customer server provides customer information, SLA terms, status checks, and contact lists.
Watch how error responses provide helpful recovery suggestions.

In [ ]:
# Import customer server functions
# For Google Colab: files are uploaded to root directory (no 'servers' folder)
from customer_server import lookup_customer, check_customer_status, get_sla_terms, list_customer_contacts

# Example 1: Look up an existing customer (SUCCESS)
print("=" * 60)
print("Example 1: Looking up existing customer CUST-001")
print("=" * 60)
customer = lookup_customer(customer_id="CUST-001")
print(f"Customer: {customer['company_name']}")
print(f"Tier: {customer['tier']}")
print(f"Status: {customer['status']}")
print(f"Account Manager: {customer['account_manager']}")

# Example 2: Try to look up non-existent customer (ERROR WITH HINTS)
print("\n" + "=" * 60)
print("Example 2: Attempting to look up non-existent customer CUST-999")
print("=" * 60)
error_response = lookup_customer(customer_id="CUST-999")
print("This customer doesn't exist. Here's the error response:\n")
import json
print(json.dumps(error_response, indent=2))
print("\n💡 The LLM can use these hints to try a different approach!")

In [ ]:
# Example 3: Get SLA terms for existing customer (SUCCESS)
print("\n" + "=" * 60)
print("Example 3: Getting SLA terms for CUST-001")
print("=" * 60)
sla = get_sla_terms("CUST-001")
print(f"SLA for {sla['company_name']}:")
print(f"  Level: {sla['sla_terms']['level']}")
print(f"  Response Time: {sla['sla_terms']['response_time_hours']} hours")
print(f"  Resolution Time: {sla['sla_terms']['resolution_time_hours']} hours")
print(f"  Support Hours: {sla['sla_terms']['support_hours']}")

### Billing Server Functions

The billing server manages invoices, payment status, billing history, and outstanding balances.
Errors include suggestions for alternative queries.

In [ ]:
# Import billing server functions
# For Google Colab: files are uploaded to root directory (no 'servers' folder)
from billing_server import get_invoice, check_payment_status, calculate_outstanding_balance

# Example 1: Get invoices for an existing customer (SUCCESS)
print("=" * 60)
print("Example 1: Getting invoices for customer CUST-001")
print("=" * 60)
invoices = get_invoice(customer_id="CUST-001")
print(f"Total invoices for customer: {invoices['total_invoices']}\n")
for inv in invoices['invoices'][:3]:  # Show first 3
    print(f"  {inv['invoice_id']}: ${inv['amount']} - {inv['status']}")

# Example 2: Try to get invoice with invalid ID (ERROR WITH HINTS)
print("\n" + "=" * 60)
print("Example 2: Attempting to get invoice INV-9999 (doesn't exist)")
print("=" * 60)
error_response = get_invoice(invoice_id="INV-9999")
print("This invoice doesn't exist. Here's the error response:\n")
import json
print(json.dumps(error_response, indent=2))
print("\n💡 Notice how the error suggests using customer_id instead!")

In [ ]:
# Example 3: Calculate outstanding balance (SUCCESS)
print("\n" + "=" * 60)
print("Example 3: Calculating outstanding balance for CUST-002")
print("=" * 60)
balance = calculate_outstanding_balance("CUST-002")
print(f"Outstanding Balance for Customer:")
print(f"  Total: ${balance['outstanding_balance']}")
print(f"  Overdue: ${balance['overdue_amount']}")
print(f"  Unpaid Invoices: {balance['number_of_unpaid_invoices']}")

### Knowledge Base Server Functions

The KB server provides article search, retrieval, related articles, and common fixes.
Error responses guide the LLM toward successful searches.

In [ ]:
# Import knowledge base server functions
# For Google Colab: files are uploaded to root directory (no 'servers' folder)
from kb_server import search_solutions, get_article

# Example 1: Search for BSOD articles (SUCCESS)
print("=" * 60)
print("Example 1: Searching for articles about BSOD")
print("=" * 60)
results = search_solutions("BSOD", limit=3)
print(f"Found {results['total_count']} articles about BSOD:\n")
for article in results['results']:
    print(f"  {article['article_id']}: {article['title']}")
    print(f"    Relevance: {article['relevance_score']}, Views: {article['views']}")

# Example 2: Search with no results (EMPTY RESULT - NOT AN ERROR)
print("\n" + "=" * 60)
print("Example 2: Searching for articles about 'xyz123nonexistent'")
print("=" * 60)
no_results = search_solutions("xyz123nonexistent", limit=3)
print(f"Found {no_results['total_count']} articles.")
print("Note: No error - just empty results. LLM can try different search terms.")

In [ ]:
# Example 3: Get an existing article (SUCCESS)
print("\n" + "=" * 60)
print("Example 3: Getting full article KB-001")
print("=" * 60)
article = get_article("KB-001")
print(f"Article: {article['title']}")
print(f"Category: {article['category']}")
print(f"\nContent Preview:")
print(article['content'][:200] + "...")

# Example 4: Try to get non-existent article (ERROR WITH HINTS)
print("\n" + "=" * 60)
print("Example 4: Attempting to get non-existent article KB-999")
print("=" * 60)
error_response = get_article("KB-999")
print("This article doesn't exist. Here's the error response:\n")
import json
print(json.dumps(error_response, indent=2))
print("\n💡 The error suggests using search_solutions to find relevant articles!")

### Asset Server Functions

The asset server tracks hardware/software assets, warranties, licenses, and asset history.
Watch how errors provide context about what data is available.

In [ ]:
# Import asset server functions
# For Google Colab: files are uploaded to root directory (no 'servers' folder)
from asset_server import lookup_asset, check_warranty

# Example 1: Look up an existing asset (SUCCESS)
print("=" * 60)
print("Example 1: Looking up asset AST-SRV-001")
print("=" * 60)
asset = lookup_asset(asset_id="AST-SRV-001")
print(f"Asset: {asset['hostname']}")
print(f"Type: {asset['asset_type']}")
print(f"Manufacturer: {asset['manufacturer']} {asset['model']}")
print(f"Location: {asset['location']}")

# Example 2: Try to look up non-existent asset (ERROR WITH HINTS)
print("\n" + "=" * 60)
print("Example 2: Attempting to look up non-existent asset AST-999")
print("=" * 60)
error_response = lookup_asset(asset_id="AST-999")
print("This asset doesn't exist. Here's the error response:\n")
import json
print(json.dumps(error_response, indent=2))
print("\n💡 Error provides context and suggests alternative approaches!")

In [ ]:
# Example 3: Check warranty for existing asset (SUCCESS)
print("\n" + "=" * 60)
print("Example 3: Checking warranty for asset AST-SRV-001")
print("=" * 60)
warranty = check_warranty("AST-SRV-001")
w = warranty['warranty']
print(f"Warranty for {warranty['hostname']}:")
print(f"  Coverage: {w['coverage_type']}")
print(f"  Status: {w['status']}")
print(f"  End Date: {w['end_date']}")
print(f"  Days Remaining: {w['remaining_days']}")
print(f"  Expired: {w['is_expired']}")

## Summary

This notebook demonstrated **Direct Function Calls** and **LLM-Friendly Error Handling**:

### What We Learned:

✅ **Successful Queries**
- All server functions can be imported and used as regular Python functions
- Perfect for testing, debugging, and simple data access in notebooks
- No async/await required
- No MCP infrastructure needed

✅ **Error Responses with Hints**
- When data doesn't exist, functions return structured error objects
- Errors include `suggested_actions` that guide the LLM toward recovery
- `follow_up_tools` recommend alternative tools to try
- `retryable` flag indicates if the query can be retried with different parameters
- These hints help gpt-5-nano make intelligent decisions when queries fail

### Key Takeaways:

1. **Error responses are designed for LLMs** - They provide actionable guidance, not just error messages
2. **Recovery strategies are built-in** - The LLM knows which tool to call next
3. **Context is preserved** - Error responses include the original query context
4. **Works perfectly in Jupyter/Colab** - No process management issues

### Using with OpenAI gpt-5-nano:

**Note:** The full MCP orchestrator with OpenAI integration does NOT work in Jupyter/Colab because Jupyter doesn't support stdin/stdout file descriptors for subprocesses (you'll get a `fileno` error).

To use natural language queries with OpenAI gpt-5-nano, run the orchestrator from regular Python:
```bash
python interactive_client.py
```

When you use `interactive_client.py`, gpt-5-nano receives these same error responses and uses the hints to:
- Try alternative search queries
- Call different tools
- Combine information from multiple sources
- Provide helpful feedback to users

See `interactive_client.py` and `test_intents.py` for examples of the full MCP orchestrator with OpenAI integration.